In [1]:
# invalid value encountered in log, infinite loop -> why?
# refactor sampling_Z (Iin, Iout, Iout_count, Iout_track)
# BaseObject, inferface with sktime

In [2]:
import math

import numpy as np

from sktime.annotation.hidalgo import Hidalgo

# generate dataset
N = 10
N_half = int(N / 2)
np.random.seed(10002)
X = np.zeros((N, 6))

# half the points from one generating regime
for j in range(1):
    X[:N_half, j] = np.random.normal(0, 3, N_half)

# the other half from another
for j in range(3):
    X[N_half:, j] = np.random.normal(2, 1, N_half)

In [3]:
# get model
K = 2
model = Hidalgo(K=K, Niter=10, seed=1)

In [4]:
out = model._fit(X)

In [5]:
out

array([[  0.72269469,   3.20993546,   0.59599049,   0.40400951,
          0.8       ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   1.        , -17.9650666 ,
         -4.50125762],
       [  0.85025968,   2.32993642,   0.80166206,   0.19833794,
          0.8       ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   0.        , -19.21897121,
         -7.78773922]])

In [3]:
model.fit(X)

In [4]:
model.d_

array([2.427715 , 1.2285935])

In [5]:
model.p_

array([0.383065, 0.616935])

In [6]:
model.Z

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1], dtype=int64)

In [7]:
X

array([[ 3.6855303 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.52647443,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 4.83614214,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.08946519,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 2.00020383,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.55822906,  1.9518051 ,  1.47156164,  0.        ,  0.        ,
         0.        ],
       [ 0.83013883,  2.46161565,  2.4118206 ,  0.        ,  0.        ,
         0.        ],
       [ 2.52227595,  1.27491395,  2.44967378,  0.        ,  0.        ,
         0.        ],
       [ 1.81087737,  2.52350174,  1.68510737,  0.        ,  0.        ,
         0.        ],
       [ 1.06737808,  0.65418774,  1.61884896,  0.        ,  0.        ,
         0.        ]])

In [13]:
model.Iin

array([2, 4, 7, 3, 9, 4, 0, 4, 7, 1, 4, 9, 0, 9, 3, 8, 6, 9, 8, 5, 9, 5,
       8, 9, 5, 6, 7, 5, 7, 4])

In [93]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(
    n_neighbors=model.q + 1, algorithm="ball_tree", metric=model.metric
).fit(X)
distances, Iin = nbrs.kneighbors(X)

nbrmat = np.zeros((model.N, model.N))
for n in range(model.q):
    nbrmat[Iin[:, 0], Iin[:, n + 1]] = 1

In [17]:
nbrmat

array([[0., 0., 1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 1., 0., 0.]])

In [95]:
Iin[:, 0]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [97]:
n = 0
Iin[:, n + 1]

array([2, 3, 0, 1, 0, 8, 8, 5, 5, 5], dtype=int64)

In [100]:
[nbrmat[row][col] = 1 for row,col in zip(Iin[:, 0], Iin[:, n + 1])]

SyntaxError: invalid syntax (2343662732.py, line 1)

In [ ]:
[if int(self.Iout[self.Iout_track[i] + j]) > -1 and Z[int(self.Iout[self.Iout_track[i] + j])] == k1 for j in range(int(self.Iout_count[i]))]

In [102]:
bool(model.use_Potts)

True

In [21]:
# Iout_count : how many neighbours
np.sum(nbrmat, axis=0)

array([2., 1., 1., 2., 5., 4., 2., 4., 3., 6.])

In [30]:
# Iout_track : cumulative sum of number of neighbours
np.cumsum(model.Iout_count)

array([ 2,  3,  4,  6, 11, 15, 17, 21, 24, 30], dtype=int32)

In [33]:
model.Iout_track

array([ 0,  2,  3,  4,  6, 11, 15, 17, 21, 24])

In [31]:
# Iout
np.where(nbrmat.T)

(array([0, 0, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 7, 7, 8,
        8, 8, 9, 9, 9, 9, 9, 9], dtype=int64),
 array([2, 4, 3, 0, 1, 4, 0, 1, 2, 3, 9, 6, 7, 8, 9, 5, 8, 0, 2, 8, 9, 5,
        6, 7, 1, 3, 4, 5, 6, 7], dtype=int64))

In [32]:
nbrmat.T

array([[0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 1., 0., 1., 1., 1., 1., 1., 0., 0.]])

In [9]:
model.Iout

array([2, 4, 3, 0, 1, 4, 0, 1, 2, 3, 9, 6, 7, 8, 9, 5, 8, 0, 2, 8, 9, 5,
       6, 7, 1, 3, 4, 5, 6, 7])

In [10]:
model.Iout_count

array([2, 1, 1, 2, 5, 4, 2, 4, 3, 6])

In [11]:
model.Iout_track

array([ 0,  2,  3,  4,  6, 11, 15, 17, 21, 24])

In [7]:
K = 2
model = Hidalgo(K=K, Niter=500, seed=10)

In [8]:
model.fit(X)

In [9]:
model.Z

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1], dtype=int64)

In [10]:
model.Pi

array([[0.312, 0.184, 0.304, 0.192, 0.264, 0.84 , 0.824, 0.808, 0.848,
        0.776],
       [0.688, 0.816, 0.696, 0.808, 0.736, 0.16 , 0.176, 0.192, 0.152,
        0.224]])

In [6]:
model.mu

array([1.464722  , 5.40974013, 2.464722  , 4.78175011, 1.17460641,
       1.95890874, 1.05863706, 1.06313386, 1.85040636, 1.28065266])

In [10]:
model.Z 

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1])

In [18]:
a = np.array([model.mu, model.Z])

In [43]:
[model.mu[i] for k in [1,2] for i in range(10) if model.Z[i] ==k]

[1.9589087445409834,
 1.058637057660818,
 1.0631338562261194,
 1.8504063601074203,
 1.2806526635220237,
 1.4647219996062466,
 5.409740134317853,
 2.4647219996062466,
 4.7817501050892925,
 1.1746064081642558]

In [59]:
[sum(model.mu[[model.Z[i] ==k for i in range(10) ]]) for k in [1,2]]

[7.211738682057365, 15.295540646783893]

[False, False, False, False, False, True, True, True, True, True]

In [61]:
[sum(np.log(model.mu[[model.Z[i] ==k for i in range(10) ]])) for k in [1,2]]

[1.6533659662674967, 4.697685272525149]

In [62]:
model.Z

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1])

In [66]:
Z = model.Z -1

In [67]:
[sum(np.log(model.mu[[Z[i] == k for i in range(model.N)]])) for k in range(model.K)]

[1.6533659662674967, 4.697685272525149]

In [69]:
[k for k in range(model.K)]

[0, 1]

In [76]:
[sum([Z[i] == k for i in range(model.N)]) for k in range(model.K)]

[5, 5]

In [78]:
model.update_zeta_prior(Z)

(24, array([25.,  7.]))

In [88]:


sum([Z[model.Iin[model.q * i + j]] == Z[i] for j in range(model.q) for i in range(model.N)])



24

In [85]:
i=0
j=0

sum(z[model.Iin[model.q * i + j]]==Z[0])

TypeError: 'numpy.bool_' object is not iterable

In [89]:
nbrmat = np.zeros((N, N))

In [90]:
nbrmat

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [92]:
Iin

NameError: name 'Iin' is not defined